In [ ]:
import json
import pandas as pd 

with open('../input_data/data_presidentielle_2022_nspp.json', 'r', encoding="utf-8") as file:
    liste_sondages_nspp = json.load(file)

In [ ]:
first_tour_scores = []

for index in range(3): #(len(liste_sondages_nspp)):
    sondage = liste_sondages_nspp[index]
    nom_institut = sondage["nom_institut"]
    debut_enquete = sondage["debut_enquete"]
    fin_enquete = sondage["fin_enquete"]
    echantillon = sondage["echantillon"]
    tours = sondage["tours"]
    for tour in tours:
        if tour['tour'] == 'Premier tour': #we restrict ourselves to the first round
            # print(tour)
            for hypothesis in tour['hypotheses']:
                    hypo_name = hypothesis['hypothese']
                    scores = []
                    for candidat in hypothesis['candidats']:
                        candidate_info = {
                            'candidat': candidat['candidat'],
                            'parti': candidat['parti'],
                            'intentions': candidat['intentions']
                        }
                        scores.append(candidate_info)
                    first_tour_scores.append({'hypothese': hypo_name, 'scores': scores})

first_tour_scores

# 1st version: with candidates' names


In [ ]:
unique_candidates = set()
for tour in first_tour_scores:
    for candidate in tour['scores']:
        unique_candidates.add(candidate["candidat"])
unique_candidates

In [ ]:

rows_candidates = []
for tour in first_tour_scores:
    row = {'Hypothese': tour['hypothese']}
    for candidate in unique_candidates:
        row[candidate] = None
    for candidate in tour['scores']:
        #print(candidate, candidate["candidat"])
        if candidate["candidat"] in unique_candidates:
            row[candidate["candidat"]] = candidate['intentions']
    rows_candidates.append(row)

rows_candidates

#TODO regrouper candidats par parti 

In [ ]:
df_candidates = pd.DataFrame(rows_candidates)
df_candidates['Total'] = df_candidates[list(unique_candidates)].sum(axis=1)
df_candidates

# 2nd version: with party's names

In [ ]:
unique_parties = set()
for tour in first_tour_scores:
    for candidate in tour['scores']:
        for party in candidate['parti']:
            if party != '':
                unique_parties.add(party)
unique_parties

In [ ]:
link_candidate_party = {'Arnaud Montebourg': 'Montebourg', 'Jean Lassalle':'Résistons', 'Eric Zemmour':'Reconquête', 'Christiane Taubira':'Taubira'} #"small" candidates like Montebourg or Taubira are assigned their own party
for key,value in link_candidate_party.items():
    unique_parties.add(value)
unique_parties

# but issue when several candidates from a same party (Montebourg, Taubira and Hidalgo for the Parti socialiste TODO) choix qui peut etre fait : les regrouper ensemble

In [ ]:

rows_parties = []
for tour in first_tour_scores:
    row = {'Hypothese': tour['hypothese']}
    for party in unique_parties:
        row[party] = None
    for candidate in tour['scores']:
        if candidate["parti"] == ['']:
            print(candidate, candidate["candidat"])
            print("no party")
            candidate["parti"] = [link_candidate_party[candidate["candidat"]]]
        for party in candidate['parti']: 
            if party in unique_parties:
                row[party] = candidate['intentions']
    rows_parties.append(row)

#rows_parties

In [ ]:
df_parties = pd.DataFrame(rows_parties)
df_parties['Total'] = df_parties[list(unique_parties)].sum(axis=1)
df_parties

In [ ]:



#df_parties.to_csv('first_tour_scores.csv', index=False, encoding='utf-8')